In [11]:
import math
import time
import numpy as np
from numba import cuda, jit, float64

TPB = 32 # thread per block

def cpu_mat_mul(A, B, C):
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            summation = 0
            for k in range(A.shape[1]):
                summation += A[i, k] * B[k, j]
            C[i, j] = summation

@jit
def cpu_mat_mul_jit(A, B, C):
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            summation = 0
            for k in range(A.shape[1]):
                summation += A[i, k] * B[k, j]
            C[i, j] = summation

@cuda.jit
def mat_mul_naive_kernal(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        summation = 0
        for k in range(A.shape[1]):
            summation += A[i, k] * B[k, j]
        C[i, j] = summation

@cuda.jit
def mat_mul_shared_kernal(A, B, C):
    s_A = cuda.shared.array((TPB, TPB), dtype=float64)  # s_ --> shared
    s_B = cuda.shared.array((TPB, TPB), dtype=float64)
    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bw = cuda.blockDim.x
    bh = cuda.blockDim.y
    #print((x, y), (tx, ty), (bx, by), (bw, bh))

    if x >= C.shape[0] or y >= C.shape[1]:
        return

    tmp = 0
    for i in range(int(A.shape[1]/TPB)):
        #print((x, y), (tx, ty), i)
        s_A[tx, ty] = A[x, ty + bw*i]
        s_B[tx, ty] = B[tx + bh*i, y]
        cuda.syncthreads()

        for j in range(TPB):
            tmp += s_A[tx, j] * s_B[j, ty]

        cuda.syncthreads()
    C[x, y] = tmp


def host_naive(A, B, C):
    d_A = cuda.to_device(A)  # d_ --> device
    d_B = cuda.to_device(B)
    d_C = cuda.device_array(C.shape, np.float64)

    threadsperblock = (TPB, TPB)
    blockspergrid_x = math.ceil(A.shape[0]/threadsperblock[0])
    blockspergrid_y = math.ceil(B.shape[1]/threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    mat_mul_naive_kernal[blockspergrid, threadsperblock](d_A, d_B, d_C)

    return d_C.copy_to_host()


def host_optimized(A, B, C):
    d_A = cuda.to_device(A)  # d_ --> device
    d_B = cuda.to_device(B)
    d_C = cuda.device_array(C.shape, np.float64)

    threadsperblock = (TPB, TPB)
    blockspergrid_x = math.ceil(A.shape[0]/threadsperblock[0])
    blockspergrid_y = math.ceil(B.shape[1]/threadsperblock[1])
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    mat_mul_shared_kernal[blockspergrid, threadsperblock](d_A, d_B, d_C)

    return d_C.copy_to_host()

def main():
    A = np.full((TPB*10, TPB*10), 0.5, dtype=np.float64)
    B = np.full((TPB*10, TPB*10), 2, dtype=np.float64)
    C = np.full((TPB*10, TPB*10), 0, dtype=np.float64)

    start = time.time()
    cpu_mat_mul(A, B, C)
    print('cpu mat mul:', time.time()-start)

    start = time.time()
    cpu_mat_mul_jit(A, B, C)
    print('cpu mat mul with numba.jit:', time.time()-start)

    start = time.time()
    ans = host_naive(A, B, C)
    print('gpu mat mul global:', time.time()-start)
    print(ans)
    
    start = time.time()
    ans = host_optimized(A, B, C)
    print('gpu mat mul shared:', time.time()-start)
    print(ans)

    
    if __name__ == '__main__':
    main()

cpu mat mul: 2.682325601577759
cpu mat mul with numba.jit: 0.19922184944152832


CudaSupportError: ignored

In [13]:
import math
import numpy as np
from numba import cuda, jit
from datetime import datetime

CPU_tmp = 0 
GPU_tmp = 0 
CPU_time = []
CPU_matsize = []
GPU_time = []
GPU_matsize = []

def info(TBS): 
  A = np.random.uniform (-100, 100, TBS*TBS*TBS*TBS)
  A = A.reshape(TBS*TBS, TBS*TBS)
  B = np.random.uniform (-100, 100, TBS*TBS*TBS*TBS)
  B = B.reshape(TBS*TBS, TBS*TBS)
  C = np.zeros((TBS*TBS,TBS*TBS ))


  def MatMul(A, B, C):
    for i in range(C.shape[0]):
      for j in range(C.shape[1]):
        sum = 0
        for k in range (A.shape[1]):
          sum += A[i,k] * B[k, j]
          C[i,j] = sum
    
  start = datetime.now()
  MatMul(A, B, C)
  CPU_res = C
  print("CPU MatMul's time: ", datetime.now()-start)
  CPU_tmp=(datetime.now()-start) 
  CPU_time.append(CPU_tmp.total_seconds()) 
  CPU_matsize.append(TBS*TBS*TBS*TBS)

  @cuda.jit 
  def MatMul_cuda(A, B, C):
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
      sum = 0
      for k in range(A.shape[1]):
        sum += A[i, k] * B[k, j]
        C[i, j] = sum

      
  def host_naive(A,B,C): 
    d_A = cuda.to_device(A) 
    d_B = cuda.to_device(B)
    d_C = cuda.device_array(C.shape, np.float64)

    block = (TBS, TBS)
    blockgrid_x = math.ceil(A.shape[0]/ block[0])
    blockgrid_y = math.ceil(B.shape[1]/ block[1])
    blockgrid = (blockgrid_x, blockgrid_y)
    

    MatMul_cuda[blockgrid, block](d_A, d_B, d_C)

    return d_C.copy_to_host()

  start = datetime.now()
  host_naive(A,B,C)
  GPU_res = C
  print("GPU MatMul's time: ", datetime.now() - start)
  GPU_tmp=(datetime.now()-start)
  GPU_time.append(GPU_tmp.total_seconds())
  GPU_matsize.append(TBS*TBS*TBS*TBS)

  ch = True
  res = CPU_res - GPU_res
  for row in res:
    for el in row:
      if el != 0:
        ch = False
        break
  if ch:
    print('Matricies are equal')
  else:
    print ('error')

TBS = 0
while TBS <=32:
  count = 1
  print()
  print("TBS", TBS)
  TBS +=1
  while count <=3:
    print("Cycle", count)
    info(TBS)
    count +=1
print ("Max size for TPB is 32!")


TBS 0
Cycle 1
CPU MatMul's time:  0:00:00.000040


CudaSupportError: ignored